In [ ]:
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import neuropacks as packs
import numpy as np
import os
import seaborn as sns

from noise_correlations import analysis, utils

%matplotlib inline

# Load Data

## Experiment Results

In [ ]:
base_path = '/Volumes/pss/fits/neurocorr'

In [ ]:
pvc11_1_path = os.path.join(base_path, 'exp4_1_tuned_values_pvc11_65_1000_1000.h5')
pvc11_2_path = os.path.join(base_path, 'exp4_2_tuned_values_pvc11_50_1000_1000.h5')
pvc11_3_path = os.path.join(base_path, 'exp4_3_tuned_values_pvc11_75_1000_1000.h5')
ac1_path = os.path.join(base_path, 'exp4_resp_values_ac1_50_1000_1000.h5')
ret2_path = os.path.join(base_path, 'exp4_tuned_values_ret2_50_1000_1000.h5')

In [ ]:
labels = [r'\textbf{V1, 1}', r'\textbf{V1, 2}', r'\textbf{V1, 3}', r'\textbf{AC}', r'\textbf{Retina}']

In [ ]:
pvc11_1 = h5py.File(pvc11_1_path, 'r')
pvc11_2 = h5py.File(pvc11_2_path, 'r')
pvc11_3 = h5py.File(pvc11_3_path, 'r')
pvc11_results = [pvc11_1, pvc11_2, pvc11_3]
ac1 = h5py.File(ac1_path, 'r')
ret2 = h5py.File(ret2_path, 'r')

## Neural Data

In [ ]:
# Get PVC11 dataset paths
pvc11_base_path = "/Volumes/pss/data/pvc11/data/spikes_gratings"
pvc11_paths = [os.path.join(pvc11_base_path, f"data_monkey{idx}_gratings.mat") for idx in range(1, 4)]
# Get PVC11 neuropack objects
pvc11s = [packs.PVC11(data_path=path) for path in pvc11_paths]
# Get PVC11 design matrices and stimulus vectors
pvc11_Xs = [pvc11.get_response_matrix(transform=None) for pvc11 in pvc11s]
pvc11_stimuli = [pvc11.get_design_matrix(form='angle') for pvc11 in pvc11s]
# Delete non-responsive neurons
pvc11_non_responsive_neurons = [
    utils.get_nonresponsive_for_stim(X=X, stimuli=stimuli)
    for X, stimuli in zip(pvc11_Xs, pvc11_stimuli)
]
pvc11_Xs = [
    np.delete(X, non_responsive_neurons, axis=1)
    if non_responsive_neurons.sum() > 0
    else X
    for X, non_responsive_neurons in zip(pvc11_Xs, pvc11_non_responsive_neurons)
]
# Remove non-tuned neurons
pvc11_tuned_neurons = [
    utils.get_tuned_units(
        X=X,
        stimuli=stimuli,
        aggregator=np.mean,
        peak_response=2.56,
        tuning_criteria='modulation_frac',
        modulation_frac=0.6,
        variance_to_mean=10.)
    for X, stimuli in zip(pvc11_Xs, pvc11_stimuli)
]
pvc11_Xs = [
    X[:, tuned_neurons] for X, tuned_neurons in zip(pvc11_Xs, pvc11_tuned_neurons)
]

In [ ]:
# Get RET2 dataset path
ret2_path = "/Volumes/pss/data/ret2/200114_fov1_data.mat"
# Get RET2 neuropack
ret2_pack = packs.RET2(data_path=ret2_path)
# Get RET2 design matrix and stimulus vector
ret2_X = ret2_pack.get_response_matrix(cells='all', response='max')
ret2_stimuli = ret2_pack.get_design_matrix(form='angle')
# Get tuned cells
ret2_X = ret2_X[:, ret2_pack.tuned_cells]

In [ ]:
# Get AC1 dataset path
ac1_path = "/Volumes/pss/data/ac1/R32_B7_HG_ext_rsp.h5"
# Get AC1 neuropack
ac1_pack = packs.AC1(data_path=ac1_path)
# Get AC1 design matrix and stimulus vector
amplitudes = np.arange(2, 8)
ac1_X = ac1_pack.get_response_matrix(amplitudes=amplitudes)
ac1_stimuli = ac1_pack.get_design_matrix(amplitudes=amplitudes)
ac1_X_selected = utils.get_tuned_units(
    X=ac1_X,
    stimuli=ac1_stimuli,
    aggregator=np.mean,
    peak_response=1,
    variance_to_mean=10.)
ac1_X = ac1_X[:, ac1_X_selected]

# Generate optimal covariance statistics

In [ ]:
pvc11_optimal_cov_corrs = [
    analysis.get_optimal_cov_statistics(
        X=X,
        stimuli=stimuli,
        units=result['units'][:],
        stims=result['stims'][:],
        v_lfi=result['v_lfi'][:],
        dim_idxs=np.arange(1, result['units'].shape[0]),
        verbose=True
    )
    for X, stimuli, result in zip(pvc11_Xs, pvc11_stimuli, pvc11_results)
]

In [ ]:
with h5py.File('neurocorr_optimality_results.h5', 'a') as f:
    for idx in range(len(pvc11_optimal_cov_corrs)):
        f[f'pvc11_{idx+1}'] = pvc11_optimal_cov_corrs[idx]

In [ ]:
ret2_optimal_cov_corrs = analysis.get_optimal_cov_statistics(
    X=ret2_X,
    stimuli=ret2_stimuli,
    units=ret2['units'][:],
    stims=ret2['stims'][:],
    v_lfi=ret2['v_lfi'][:],
    dim_idxs=np.arange(1, ret2['units'].shape[0]),
    verbose=True
)

In [ ]:
with h5py.File('neurocorr_optimality_results.h5', 'a') as f:
    f['ret2'] = ret2_optimal_cov_corrs

In [ ]:
ac1_optimal_cov_corrs = analysis.get_optimal_cov_statistics(
    X=ac1_X,
    stimuli=ac1_stimuli,
    units=ac1['units'][:],
    stims=ac1['stims'][:],
    v_lfi=ac1['v_lfi'][:],
    dim_idxs=np.arange(1, ac1['units'].shape[0]),
    verbose=True
)

In [ ]:
with h5py.File('neurocorr_optimality_results.h5', 'a') as f:
    f['ac1'] = ac1_optimal_cov_corrs